In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
from IPython.display import clear_output
import os

from quick_pp.objects import Project, Well
from quick_pp.qaqc import badhole_flag, mask_outside_threshold, neu_den_xplot_hc_correction

In [ ]:
folder = r"C:\projects\quick_pp\notebooks\data\01_raw"
file_names = os.listdir(folder)
files = [os.path.join(folder, file) for file in file_names]

In [ ]:
project = Project("test")

In [ ]:
project._read_las(files)

In [ ]:
project._save('test')

In [ ]:
test_load = Project()._load(r"C:\projects\quick_pp\notebooks\data\04_project\test.qpp")

In [ ]:
test_load.data['X3'].description

# Lithology

In [ ]:
from quick_pp.lithology.sand_silt_clay import SandSiltClay
from quick_pp.plotter import plotly_log

In [ ]:
args = {
    # 'dry_clay_point': (0.27, 2.8)
    'silt_line_angle': 117,
    # 'wet_clay_point': (0.45, 2.45),
}
ssc_model = SandSiltClay(**args)

neu_den_df = df[['DEPTH', 'NPHI', 'RHOB']].dropna()
vsand, vsilt, vcld, vclb, xplot = ssc_model.estimate_lithology(
    neu_den_df['NPHI'], neu_den_df['RHOB'], model='kuttan_modified', xplot=True)
    # neu_den_df['NPHI'], neu_den_df['RHOB'], model='kuttan', xplot=True, **args)
df_ssc_model = pd.DataFrame(
    {'VSAND': vsand, 'VSILT': vsilt, 'VCLW': vcld + vclb, 'VCLB': vclb, 'VCLD': vcld},
    index=neu_den_df.DEPTH
)
df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
xplot

In [ ]:
fig = plotly_log(df_ssc)
fig.show()
# fig.write_html('plot.html')

## Hydrocarbon Correction

In [ ]:
# Correct NPHI and RHOB for HC
df_corr = df.copy()
nphihc, rhobhc = neu_den_xplot_hc_correction(df_corr['NPHI'], df_corr['RHOB'], gr=df_corr['GR'],
                                             dry_sand_point=(-.02, 2.65),
                                             dry_clay_point=(.33, 2.7),
                                             corr_angle=50)
df_corr[['NPHI', 'RHOB']] = pd.DataFrame({'NPHI': nphihc, 'RHOB': rhobhc}).astype(float)
neu_den_df = df_corr[['DEPTH', 'NPHI', 'RHOB']].dropna()

vsand, vsilt, vcld, vclb, xplot = ssc_model.estimate_lithology(
    neu_den_df['NPHI'], neu_den_df['RHOB'], model='kuttan_modified', xplot= True, normalize= True)
    # neu_den_df['NPHI'], neu_den_df['RHOB'], model='kuttan', xplot=True, normalize= True)
df_ssc_model = pd.DataFrame(
    {'VSAND': vsand, 'VSILT': vsilt, 'VCLW': vcld + vclb, 'VCLB': vclb, 'VCLD': vcld},
    index=neu_den_df.DEPTH
)
df_ssc_hc = df_corr.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
xplot

In [ ]:
fig = plotly_log(df_ssc_hc)
fig.show()
# fig.write_html('plot.html')

# Porosity

In [ ]:
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix

In [ ]:
# To compare with actual PHIT
df_ssc_hc['PHIT'] = neu_den_xplot_poro(df_ssc_hc['NPHI'], df_ssc_hc['RHOB'], 'ssc', True,
                    dry_sand_point=(-.02, 2.65),
                    dry_silt_point=(.1, 2.68),
                    dry_clay_point=(.27, 2.7))

In [ ]:
# To compare PHID with actual PHIT
vsand, vsilt, vcld, vclb, xplot = ssc_model.estimate_lithology(
    neu_den_df['NPHI'], neu_den_df['RHOB'], model='kuttan_modified', normalize=False)
df_ssc_model = pd.DataFrame(
    {'VSAND': vsand, 'VSILT': vsilt, 'VCLW': vcld + vclb, 'VCLB': vclb, 'VCLD': vcld},
)
rho_ma = rho_matrix(df_ssc_model['VSAND'], df_ssc_model['VSILT'], df_ssc_model['VCLD'])

df_ssc_hc['PHID'] = density_porosity(df_ssc_hc['RHOB'], rho_ma)
df_ssc_hc.plot(x='DEPTH', y=['PHIT', 'PHID'], figsize=(20, 5))

# Water Saturation

In [ ]:
from quick_pp.saturation import *
from quick_pp.plotter import *

In [ ]:
# Waxman-Smits method
temp_grad = estimate_temperature_gradient(df_ssc_hc['DEPTH'] / 7, 'metric')  # Arbitrarily convert MD to TVD
water_salinity = 30000
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)

b = estimate_b_waxman_smits(temp_grad, rw)

qv = estimate_qv(df_ssc_hc.VCLD, df_ssc_hc.PHIT, cec_clay=.1)
# qv = estimate_qv_hill(df_ssc_model['VCLB'], df_ssc_hc.PHIT, water_salinity)
# qv = estimate_qv_lavers(df_ssc_hc.PHIT)

# m = estimate_m_archie(df_ssc_hc['RT'], rw, df_ssc_hc['PHIT'])
# m = estimate_m_indonesian(df_ssc_hc['RT'], rw, df_ssc_hc['PHIT'], df_ssc_hc['VCLD'], 50)
m = 2

df_ssc_hc['SWT'] = waxman_smits_saturation(df_ssc_hc['RT'], rw, df_ssc_hc['PHIT'], qv, b, m)

# Archie method
df_ssc_hc['SWT_A'] = archie_saturation(df_ssc_hc['RT'], rw, df_ssc_hc['PHIT'], 1, 2, 2)

df_ssc_hc.plot(x='DEPTH', y=['SWT', 'SWT_A'], figsize=(20, 5))

# Permeability

In [ ]:
from quick_pp.permeability import *

In [ ]:
constant = df_ssc_hc['VCLB']**1.75
Swirr = constant / df_ssc_hc['PHIT']
df_ssc_hc['PERM'] = choo_permeability(df_ssc_hc['VCLW'], df_ssc_hc['VSILT'], df_ssc_hc['PHIT'])
df_ssc_hc['PERM_KC'] = kozeny_carman_permeability(df_ssc_hc['PHIT'], Swirr)
df_ssc_hc['PERM_T'] = timur_permeability(df_ssc_hc['PHIT'], Swirr)
df_ssc_hc['PERM_C'] = coates_permeability(df_ssc_hc['PHIT'], Swirr)
df_ssc_hc['PERM_TX'] = tixier_permeability(df_ssc_hc['PHIT'], Swirr)
df_ssc_hc.plot(x='DEPTH', y=['PERM', 'PERM_KC', 'PERM_T', 'PERM_C', 'PERM_TX'], figsize=(20, 5), logy=True)

# Reservoir Summary

In [ ]:
from quick_pp.ressum import *

In [ ]:
df_ssc_hc['ZONES'] = 'ALL'
depth = df_ssc_hc.DEPTH
vshale = df_ssc_hc.VCLW
phit = df_ssc_hc.PHIT
swt = df_ssc_hc.SWT
perm = df_ssc_hc.PERM
zones = df_ssc_hc.ZONES
ressum_df = calc_reservoir_summary(depth, vshale, phit, swt, perm, zones, cutoffs=[0.3, 0.1, .75])
ressum_df

In [ ]:
fig = plotly_log(df_ssc_hc)
fig.show()
# fig.write_html('plot.html')

# Rock Type

In [ ]:
std = df_ssc_hc['VCLW'].describe()['std']
standard_q = [0.05, 0.15, 0.5]
proportion = [pct - std for pct in standard_q]
proportion = standard_q if any([p < 0.15 for p in proportion]) else proportion
q_dict = df_ssc_hc['VCLW'].quantile(proportion).to_dict()
q_dict

In [ ]:
df_ssc_hc['ROCK_TYPE'] = np.where(df_ssc_hc['VCLW'] < list(q_dict.values())[0], 1,
                               np.where(df_ssc_hc['VCLW'] < list(q_dict.values())[1], 2,
                                        np.where(df_ssc_hc['VCLW'] < list(q_dict.values())[2], 3, 4)))

In [ ]:
df_ssc_hc.plot(y='PERM', x='PHIT', logy=True, kind='scatter', figsize=(5, 5), c='ROCK_TYPE', colormap='viridis')